In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
df = pd.read_csv("../processed/pl_to_20180924.csv", 
                usecols=['season', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 
                         'FTR', 'HS', 'AS', 'HST', 'AST'],
                parse_dates=['Date'],
                dtype={"season": "str"})

In [4]:
df.head()

,AS,AST,AwayTeam,Date,FTAG,FTHG,FTR,HS,HST,HomeTeam,season
0,13.0,6.0,Bolton,2005-08-13,2.0,2.0,D,3.0,2.0,Aston Villa,0506
1,12.0,5.0,Man United,2005-08-13,2.0,0.0,A,10.0,5.0,Everton,0506
2,7.0,4.0,Birmingham,2005-08-13,0.0,0.0,D,15.0,7.0,Fulham,0506
3,13.0,3.0,West Brom,2005-08-13,0.0,0.0,D,15.0,8.0,Man City,0506
4,16.0,7.0,Liverpool,2005-08-13,0.0,0.0,D,4.0,2.0,Middlesbrough,0506


In [5]:
len(df)

6520

In [6]:
df.dtypes

AS                 float64
AST                float64
AwayTeam            object
Date        datetime64[ns]
FTAG               float64
FTHG               float64
FTR                 object
HS                 float64
HST                float64
HomeTeam            object
season              object
dtype: object

Thinking now, given the problems with teams disappearing and reappearing through relegation and promotion, we may be better off treating each season on its own, the concatenating together

In [134]:
df_a = pd.read_csv("../data/16.csv")

In [147]:
df_a.columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC',
       'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD',
       'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA',
       'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'Bb1X2', 'BbMxH', 'BbAvH',
       'BbMxD', 'BbAvD', 'BbMxA', 'BbAvA', 'BbOU', 'BbMx>2.5', 'BbAv>2.5',
       'BbMx<2.5', 'BbAv<2.5', 'BbAH', 'BbAHh', 'BbMxAHH', 'BbAvAHH',
       'BbMxAHA', 'BbAvAHA', 'PSCH', 'PSCD', 'PSCA'],
      dtype='object')

In [168]:
df = pd.read_csv("../data/16.csv", usecols=['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 
                         'FTR', 'HS', 'AS', 'HST', 'AST', 'BbMxH', 'BbMxD', 'BbMxA', 'FTR'], parse_dates=['Date'], dayfirst=True)

In [169]:
df.sort_values(["Date"], inplace=True)

In [7]:
cols = ['Date', 'HomeTeam', 'AwayTeam', 
        'FTHG', 'FTAG', 'HS', 'AS', 'HST', 'AST',
        'FTR',
        'season']
bet_cols = ['BbMxH', 'BbMxD', 'BbMxA']

In [8]:
df_list = []
cols = ['Date', 'HomeTeam', 'AwayTeam', 
        'FTHG', 'FTAG', 'HS', 'AS', 'HST', 'AST',
        'FTR']
bet_cols = ['BbMxH', 'BbMxD', 'BbMxA']
for file in sorted(os.listdir("../data/")):
    if file == "01.csv" or file == "02.csv" or file == "03.csv" or file == "04.csv" or file == "05.csv":
        print("reading " + file)
        df = pd.read_csv("../data/" + file, 
                         usecols= cols,
                         parse_dates=['Date'], 
                         dayfirst=True)
    elif file != "01.csv":
        print("reading " + file)
        df = pd.read_csv("../data/" + file, 
                         usecols= cols + bet_cols,
                         parse_dates=['Date'], 
                         dayfirst=True)
        
    for i, row in enumerate(df.itertuples()):
        
        home_team = row.HomeTeam
        away_team = row.AwayTeam
        match_date = row.Date

        hr = df[((df.HomeTeam == home_team) | (df.AwayTeam == home_team)) & 
                    (df.Date < match_date)].tail(6)
        ar = df[((df.HomeTeam == away_team) | (df.AwayTeam == away_team)) & 
                    (df.Date < match_date)].tail(6)
        if (len(hr) >= 6) & (len(ar) >= 6):
            htgf_last_6 = hr[hr.HomeTeam == home_team].FTHG.sum() + hr[hr.AwayTeam == home_team].FTAG.sum()
            htga_last_6 = hr[hr.HomeTeam == home_team].FTAG.sum() + hr[hr.AwayTeam == home_team].FTHG.sum()
            htsf_last_6 = hr[hr.HomeTeam == home_team].HS.sum() + hr[hr.AwayTeam == home_team].AS.sum()
            htsa_last_6 = hr[hr.HomeTeam == home_team].AS.sum() + hr[hr.AwayTeam == home_team].HS.sum()
            htstf_last_6 = hr[hr.HomeTeam == home_team].HST.sum() + hr[hr.AwayTeam == home_team].AST.sum()
            htsta_last_6 = hr[hr.HomeTeam == home_team].AST.sum() + hr[hr.AwayTeam == home_team].HST.sum()

            atgf_last_6 = ar[ar.HomeTeam == away_team].FTHG.sum() + ar[ar.AwayTeam == away_team].FTAG.sum()
            atga_last_6 = ar[ar.HomeTeam == away_team].FTAG.sum() + ar[ar.AwayTeam == away_team].FTHG.sum()
            atsf_last_6 = ar[ar.HomeTeam == away_team].HS.sum() + ar[ar.AwayTeam == away_team].AS.sum()
            atsa_last_6 = ar[ar.HomeTeam == away_team].AS.sum() + ar[ar.AwayTeam == away_team].HS.sum()
            atstf_last_6 = ar[ar.HomeTeam == away_team].HST.sum() + ar[ar.AwayTeam == away_team].AST.sum()
            atsta_last_6 = ar[ar.HomeTeam == away_team].AST.sum() + ar[ar.AwayTeam == away_team].HST.sum()

            df.loc[i, "htgf_last_6"] = htgf_last_6
            df.loc[i, "htga_last_6"] = htga_last_6
            df.loc[i, "htsf_last_6"] = htsf_last_6
            df.loc[i, "htsa_last_6"] = htsa_last_6
            df.loc[i, "htstf_last_6"] = htstf_last_6
            df.loc[i, "htsta_last_6"] = htsta_last_6

            df.loc[i, "atgf_last_6"] = atgf_last_6
            df.loc[i, "atga_last_6"] = atga_last_6
            df.loc[i, "atsf_last_6"] = atsf_last_6
            df.loc[i, "atsa_last_6"] = atsa_last_6
            df.loc[i, "atstf_last_6"] = atstf_last_6
            df.loc[i, "atsta_last_6"] = atsta_last_6

    df_list.append(df)

reading 01.csv
reading 02.csv
reading 03.csv
reading 04.csv
reading 05.csv
reading 06.csv
reading 07.csv
reading 08.csv
reading 09.csv
reading 10.csv
reading 11.csv
reading 12.csv
reading 13.csv
reading 14.csv
reading 15.csv
reading 16.csv
reading 17.csv
reading 18.csv
reading 19.csv


In [9]:
result = pd.concat(df_list).sort_values("Date")

In [26]:
result.dropna(subset=['atsta_last_6'], inplace=True)

In [27]:
result.columns

Index(['AS', 'AST', 'AwayTeam', 'BbMxA', 'BbMxD', 'BbMxH', 'Date', 'FTAG',
       'FTHG', 'FTR', 'HS', 'HST', 'HomeTeam', 'atga_last_6', 'atgf_last_6',
       'atsa_last_6', 'atsf_last_6', 'atsta_last_6', 'atstf_last_6',
       'htga_last_6', 'htgf_last_6', 'htsa_last_6', 'htsf_last_6',
       'htsta_last_6', 'htstf_last_6'],
      dtype='object')

In [28]:
result_features_labels = result[['Date', 'atga_last_6', 'atgf_last_6',
       'atsa_last_6', 'atsf_last_6', 'atsta_last_6', 'atstf_last_6',
       'htga_last_6', 'htgf_last_6', 'htsa_last_6', 'htsf_last_6',
       'htsta_last_6', 'htstf_last_6', 'FTR']]

In [29]:
train = result_features_labels[result_features_labels.Date < pd.datetime(2014, 6, 1)]

In [30]:
test = result_features_labels[result_features_labels.Date >= pd.datetime(2014, 6, 1)]

In [36]:
test.drop(['Date'], inplace=True, axis=1)

/home/alexander/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [37]:
train.drop(['Date'], inplace=True, axis=1)

/home/alexander/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [38]:
train.to_csv("../processed/train_last_6.csv")

In [39]:
test.to_csv("../processed/test_last_6.csv")

Check that in the first 228 rows of each season, each team has played 6 matches, then restrict to this

In [160]:
from collections import Counter

In [166]:
for file in sorted(os.listdir("../data/")):
    if file[0] in ['0', '1'] and file != "01.csv":
        print("reading " + file)
        temp = pd.read_csv("../data/" + file, 
                                   usecols=list(range(48)), 
                                   encoding="latin-1")
        
        print(a.most_common(1))

reading 02.csv
[('Charlton', 12)]
reading 03.csv
[('Aston Villa', 13)]
reading 04.csv
[('Fulham', 12)]
reading 05.csv
[('Aston Villa', 12)]
reading 06.csv
[('Sunderland', 13)]
reading 07.csv
[('Bolton', 12)]
reading 08.csv
[('Aston Villa', 12)]
reading 09.csv
[('Arsenal', 12)]
reading 10.csv
[('Aston Villa', 12)]
reading 11.csv
[('Aston Villa', 12)]
reading 12.csv
[('Blackburn', 12)]
reading 13.csv
[('Newcastle', 12)]
reading 14.csv
[('Liverpool', 12)]
reading 15.csv
[('Man United', 12)]
reading 16.csv
[('Chelsea', 12)]
reading 17.csv
[('Burnley', 13)]
reading 18.csv
[('Chelsea', 12)]
reading 19.csv
[('Watford', 4)]
